In [1]:
# RUSH HOUR REALITY - Data Processing & Visualization

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive mounted")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted


In [2]:
# Install Required Packages
!pip install polars plotly folium
print("Packages installed")


Packages installed


In [3]:
# Import Libraries

import polars as pl
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

print("Libraries imported")


Libraries imported


In [4]:
# Set Paths

DATA_DIR = Path("/content/drive/MyDrive/Rush-Hour-Reality/rush-hour-reality/rush_hour_data")
STATIC_DIR = Path("/content/drive/MyDrive/Rush-Hour-Reality/rush-hour-reality/gtfs_static")

print(f"Data directory: {DATA_DIR}")
print(f"Static directory: {STATIC_DIR}")

Data directory: /content/drive/MyDrive/Rush-Hour-Reality/rush-hour-reality/rush_hour_data
Static directory: /content/drive/MyDrive/Rush-Hour-Reality/rush-hour-reality/gtfs_static


In [5]:
# Load All Trip Updates Files

trip_files = sorted(DATA_DIR.glob("trip_updates_*.csv"))
print(f"Found {len(trip_files)} trip update files")

# Read with explicit schema to handle inconsistencies
schema = {
    'collection_timestamp': pl.Utf8,
    'entity_id': pl.Utf8,
    'trip_id': pl.Utf8,
    'route_id': pl.Utf8,
    'direction_id': pl.Utf8,
    'start_date': pl.Utf8,
    'start_time': pl.Utf8,
    'vehicle_id': pl.Utf8,
    'vehicle_label': pl.Utf8,
    'stop_sequence': pl.Utf8,
    'stop_id': pl.Utf8,
    'arrival_delay': pl.Utf8,
    'arrival_time': pl.Utf8,
    'departure_delay': pl.Utf8,
    'departure_time': pl.Utf8
}

df_trips = pl.concat([
    pl.read_csv(f, schema_overrides=schema, ignore_errors=True)
    for f in trip_files
])

# Convert to proper types after loading
df_trips = df_trips.with_columns([
    pl.col('arrival_delay').cast(pl.Int64, strict=False),
    pl.col('departure_delay').cast(pl.Int64, strict=False),
    pl.col('direction_id').cast(pl.Int64, strict=False),
    pl.col('stop_sequence').cast(pl.Int64, strict=False)
])

print(f"Loaded {len(df_trips):,} trip update records")

Found 358 trip update files
Loaded 4,875,839 trip update records


In [6]:
# Load All Vehicle Position Files

vehicle_files = sorted(DATA_DIR.glob("vehicle_positions_*.csv"))
print(f"Found {len(vehicle_files)} vehicle position files")

# Read with explicit schema to handle inconsistencies
schema = {
    'collection_timestamp': pl.Utf8,
    'entity_id': pl.Utf8,
    'vehicle_id': pl.Utf8,
    'vehicle_label': pl.Utf8,
    'trip_id': pl.Utf8,
    'route_id': pl.Utf8,
    'direction_id': pl.Utf8,
    'start_date': pl.Utf8,
    'start_time': pl.Utf8,
    'latitude': pl.Utf8,
    'longitude': pl.Utf8,
    'bearing': pl.Utf8,
    'speed': pl.Utf8,
    'current_stop_sequence': pl.Utf8,
    'stop_id': pl.Utf8,
    'current_status': pl.Utf8,
    'timestamp': pl.Utf8,
    'congestion_level': pl.Utf8
}

df_vehicles = pl.concat([
    pl.read_csv(f, schema_overrides=schema, ignore_errors=True)
    for f in vehicle_files
])

# Convert to proper types after loading
df_vehicles = df_vehicles.with_columns([
    pl.col('latitude').cast(pl.Float64, strict=False),
    pl.col('longitude').cast(pl.Float64, strict=False),
    pl.col('bearing').cast(pl.Float64, strict=False),
    pl.col('speed').cast(pl.Float64, strict=False),
    pl.col('direction_id').cast(pl.Int64, strict=False),
    pl.col('current_stop_sequence').cast(pl.Int64, strict=False),
    pl.col('current_status').cast(pl.Int64, strict=False),
    pl.col('timestamp').cast(pl.Int64, strict=False),
    pl.col('congestion_level').cast(pl.Int64, strict=False)
])

print(f"Loaded {len(df_vehicles):,} vehicle position records")

Found 496 vehicle position files
Loaded 390,115 vehicle position records


In [7]:
# Load Static GTFS Reference Data
df_routes = pl.read_csv(STATIC_DIR / "routes.txt")
df_trips_ref = pl.read_csv(STATIC_DIR / "trips.txt")
df_stops = pl.read_csv(STATIC_DIR / "stops.txt")

print(f"Routes: {len(df_routes):,}")
print(f"Trips: {len(df_trips_ref):,}")
print(f"Stops: {len(df_stops):,}")

Routes: 805
Trips: 187,008
Stops: 14,015


In [ ]:
# Join Real-Time with Static Data

df_with_routes = (
    df_trips
    .join(df_trips_ref.select(['trip_id', 'route_id']), on='trip_id', how='left', suffix='_ref')
    .join(df_routes.select(['route_id', 'route_short_name', 'route_long_name']),
          left_on='route_id_ref', right_on='route_id', how='left')
    .join(df_stops.select(['stop_id', 'stop_name', 'stop_lat', 'stop_lon']),
          on='stop_id', how='left')
)

print(f"Joined data: {len(df_with_routes):,} records")


In [ ]:
# Clean and Process Data

df_clean = (
    df_with_routes
    .filter(pl.col('arrival_delay').is_not_null())
    .filter(pl.col('route_short_name').is_not_null())
    .with_columns([
        pl.col('collection_timestamp').str.strptime(pl.Datetime, format='%Y-%m-%dT%H:%M:%S%z').alias('datetime'),
        (pl.col('arrival_delay') / 60.0).alias('delay_minutes')
    ])
    .with_columns([
        pl.col('datetime').dt.hour().alias('hour'),
        pl.col('datetime').dt.date().alias('date'),
        pl.col('datetime').dt.weekday().alias('weekday')
    ])
)

print(f"Cleaned data: {len(df_clean):,} records")
print(f"Date range: {df_clean['date'].min()} to {df_clean['date'].max()}")


In [ ]:
# Create Time-Based Features

df_analyzed = df_clean.with_columns([
    pl.when(pl.col('hour').is_between(7, 9))
      .then(pl.lit('Morning Rush'))
      .when(pl.col('hour').is_between(17, 19))
      .then(pl.lit('Evening Rush'))
      .otherwise(pl.lit('Off Peak'))
      .alias('time_period'),

    pl.when(pl.col('delay_minutes') <= 1)
      .then(pl.lit('On Time'))
      .when(pl.col('delay_minutes') <= 5)
      .then(pl.lit('Minor Delay'))
      .when(pl.col('delay_minutes') <= 10)
      .then(pl.lit('Moderate Delay'))
      .otherwise(pl.lit('Major Delay'))
      .alias('delay_category')
])

print(f"Added time_period and delay_category features")


In [ ]:
# Define Greater Dublin Area Bounding Box
LAT_MIN = 53.15  # South boundary (Bray)
LAT_MAX = 53.60  # North boundary (Balbriggan)
LON_MIN = -6.60  # West boundary (Maynooth/Leixlip)
LON_MAX = -6.00  # East boundary (Howth)

print(f"Original records: {len(df_analyzed):,}")

# Simple filter - keep only records within the bounding box
df_analyzed = df_analyzed.filter(
    (pl.col('stop_lat') >= LAT_MIN) &
    (pl.col('stop_lat') <= LAT_MAX) &
    (pl.col('stop_lon') >= LON_MIN) &
    (pl.col('stop_lon') <= LON_MAX)
)

print(f"Filtered records (Greater Dublin Area): {len(df_analyzed):,}")
print(f"Unique routes: {df_analyzed['route_short_name'].n_unique()}")
print(f"Unique stops: {df_analyzed['stop_id'].n_unique()}")


In [ ]:
# Save Processed Dataset
# Save the cleaned, analyzed dataset for the report
df_analyzed.write_csv('/content/drive/MyDrive/Rush-Hour-Reality/processed_data.csv')

print("Processed dataset saved")
print(f"Final dataset: {len(df_analyzed):,} records")
print(f"Location: MyDrive/Rush-Hour-Reality/processed_data.csv")


In [ ]:
#Final DF with Required Fields Preparation

import json
import polars as pl

def get_delay_severity(delay_minutes):
    """Returns color, category, and emoji based on delay"""
    if delay_minutes <= 2:
        return {'color': '#00d084', 'category': 'On Time', 'emoji': '✓', 'severity': 0}
    elif delay_minutes <= 6:
        return {'color': '#ffb700', 'category': 'Minor Delay', 'emoji': '⚠', 'severity': 1}
    elif delay_minutes <= 10:
        return {'color': '#ff6b35', 'category': 'Moderate Delay', 'emoji': '⚠', 'severity': 2}
    else:
        return {'color': '#f5576c', 'category': 'Major Delay', 'emoji': '✕', 'severity': 3}


route_data = {}

for route_short in sorted(df_analyzed['route_short_name'].unique().to_list()):
    route_all = df_analyzed.filter(pl.col('route_short_name') == route_short)
    directions = route_all['direction_id'].unique().to_list()
    route_data[route_short] = {'directions': {}}

    for direction in directions:
        direction_trips = (
            route_all
            .filter(pl.col('direction_id') == direction)
            .select(['trip_id', 'stop_sequence', 'stop_id', 'stop_lat', 'stop_lon', 'delay_minutes', 'stop_name'])
            .unique()
        )

        trip_counts = (
            direction_trips
            .group_by('trip_id')
            .agg(pl.len().alias('stop_count'))
            .sort('stop_count', descending=True)
        )

        if len(trip_counts) > 0:
            representative_trip = trip_counts['trip_id'][0]
            route_stops = (
                direction_trips
                .filter(pl.col('trip_id') == representative_trip)
                .sort('stop_sequence')
            ).to_pandas()

            if len(route_stops) > 2:
                stops_coords = []
                for _, row in route_stops.iterrows():
                    delay = row['delay_minutes']
                    severity_info = get_delay_severity(delay)
                    stops_coords.append({
                        'lat': row['stop_lat'],
                        'lon': row['stop_lon'],
                        'name': row['stop_name'],
                        'delay': round(delay, 1),
                        'color': severity_info['color'],
                        'category': severity_info['category'],
                        'severity': severity_info['severity'],
                        'emoji': severity_info['emoji']
                    })

                avg_delay = route_stops['delay_minutes'].mean()
                route_severity = get_delay_severity(avg_delay)
                first_stop = stops_coords[0]['name']
                last_stop = stops_coords[-1]['name']
                direction_name = f"{first_stop} -> {last_stop}"

                route_data[route_short]['directions'][int(direction)] = {
                    'delay': round(avg_delay, 1),
                    'color': route_severity['color'],
                    'category': route_severity['category'],
                    'severity': route_severity['severity'],
                    'stops': stops_coords,
                    'name': direction_name
                }



In [ ]:
# Get buses by area
area_buses = {
    'north': sorted(list(set(df_analyzed.filter(pl.col('stop_lat') > 53.37)['route_short_name'].unique().to_list()))),
    'central': sorted(list(set(df_analyzed.filter((pl.col('stop_lat') > 53.32) & (pl.col('stop_lat') <= 53.37))['route_short_name'].unique().to_list()))),
    'south': sorted(list(set(df_analyzed.filter(pl.col('stop_lat') <= 53.32)['route_short_name'].unique().to_list()))),
    'west': sorted(list(set(df_analyzed.filter(pl.col('stop_lon') < -6.30)['route_short_name'].unique().to_list())))
}


In [ ]:
peak_morning = df_analyzed.filter((pl.col('hour') >= 7) & (pl.col('hour') < 10))
peak_evening = df_analyzed.filter((pl.col('hour') >= 17) & (pl.col('hour') < 20))
off_peak = df_analyzed.filter(~((pl.col('hour') >= 7) & (pl.col('hour') < 10)) & ~((pl.col('hour') >= 17) & (pl.col('hour') < 20)))

peak_hours_data = {
    'morning': {
        'avg_delay': round(peak_morning['delay_minutes'].mean(), 1),
        'max_delay': round(peak_morning['delay_minutes'].max(), 1),
        'min_delay': round(peak_morning['delay_minutes'].min(), 1),
        'sample_size': len(peak_morning),
        'routes_affected': int(peak_morning['route_short_name'].n_unique()),
        'time_range': '7:00 AM - 10:00 AM'
    },
    'evening': {
        'avg_delay': round(peak_evening['delay_minutes'].mean(), 1),
        'max_delay': round(peak_evening['delay_minutes'].max(), 1),
        'min_delay': round(peak_evening['delay_minutes'].min(), 1),
        'sample_size': len(peak_evening),
        'routes_affected': int(peak_evening['route_short_name'].n_unique()),
        'time_range': '5:00 PM - 8:00 PM'
    },
    'off_peak': {
        'avg_delay': round(off_peak['delay_minutes'].mean(), 1),
        'max_delay': round(off_peak['delay_minutes'].max(), 1),
        'min_delay': round(off_peak['delay_minutes'].min(), 1),
        'sample_size': len(off_peak),
        'routes_affected': int(off_peak['route_short_name'].n_unique()),
        'time_range': 'Off Peak Hours'
    }
}


In [ ]:
# Top Problem Routes
problem_routes = (
    df_analyzed
    .group_by('route_short_name')
    .agg([
        pl.col('delay_minutes').mean().alias('avg_delay'),
        pl.col('delay_minutes').max().alias('max_delay'),
        pl.col('delay_minutes').min().alias('min_delay'),
        pl.col('route_short_name').count().alias('sample_size')
    ])
    .sort('avg_delay', descending=True)
    .head(10)
    .to_pandas()
)

top_problem_routes = []
for _, row in problem_routes.iterrows():
    top_problem_routes.append({
        'route': str(row['route_short_name']),
        'avg_delay': round(row['avg_delay'], 1),
        'max_delay': round(row['max_delay'], 1),
        'min_delay': round(row['min_delay'], 1),
        'samples': int(row['sample_size']),
        'severity': get_delay_severity(row['avg_delay'])['category']
    })


In [ ]:
# Hourly Pattern
hourly_pattern = (
    df_analyzed
    .group_by('hour')
    .agg([
        pl.col('delay_minutes').mean().alias('avg_delay'),
        pl.col('route_short_name').n_unique().alias('routes_affected'),
        pl.col('route_short_name').count().alias('sample_size')
    ])
    .sort('hour')
    .to_pandas()
)

hourly_data = []
for _, row in hourly_pattern.iterrows():
    hourly_data.append({
        'hour': int(row['hour']),
        'avg_delay': round(row['avg_delay'], 1),
        'routes_affected': int(row['routes_affected']),
        'samples': int(row['sample_size'])
    })



In [ ]:
# Geographic Data
central_data = df_analyzed.filter((pl.col('stop_lat') > 53.32) & (pl.col('stop_lat') <= 53.37))
north_data = df_analyzed.filter(pl.col('stop_lat') > 53.37)
south_data = df_analyzed.filter(pl.col('stop_lat') <= 53.32)
west_data = df_analyzed.filter(pl.col('stop_lon') < -6.30)

geographic_stats = {
    'central': {'avg_delay': round(central_data['delay_minutes'].mean(), 1), 'max_delay': round(central_data['delay_minutes'].max(), 1), 'min_delay': round(central_data['delay_minutes'].min(), 1), 'routes': len(area_buses['central']), 'label': 'Central Dublin', 'color': '#f5576c'},
    'north': {'avg_delay': round(north_data['delay_minutes'].mean(), 1), 'max_delay': round(north_data['delay_minutes'].max(), 1), 'min_delay': round(north_data['delay_minutes'].min(), 1), 'routes': len(area_buses['north']), 'label': 'North Dublin', 'color': '#ffb700'},
    'south': {'avg_delay': round(south_data['delay_minutes'].mean(), 1), 'max_delay': round(south_data['delay_minutes'].max(), 1), 'min_delay': round(south_data['delay_minutes'].min(), 1), 'routes': len(area_buses['south']), 'label': 'South Dublin', 'color': '#ff6b35'},
    'west': {'avg_delay': round(west_data['delay_minutes'].mean(), 1), 'max_delay': round(west_data['delay_minutes'].max(), 1), 'min_delay': round(west_data['delay_minutes'].min(), 1), 'routes': len(area_buses['west']), 'label': 'West Dublin', 'color': '#00d084'}
}

In [ ]:
# Overall Statistics
overall_stats = {
    'total_routes': int(df_analyzed['route_short_name'].n_unique()),
    'total_stops': int(df_analyzed['stop_id'].n_unique()),
    'overall_avg_delay': round(df_analyzed['delay_minutes'].mean(), 1),
    'max_delay': round(df_analyzed['delay_minutes'].max(), 1),
    'min_delay': round(df_analyzed['delay_minutes'].min(), 1),
    'on_time_count': int(df_analyzed.filter(pl.col('delay_minutes') <= 2).height),
    'major_delay_count': int(df_analyzed.filter(pl.col('delay_minutes') > 10).height)
}

In [ ]:
# Chart data
peak_hours_chart_data = {
    'labels': [peak_hours_data['morning']['time_range'], peak_hours_data['evening']['time_range'], peak_hours_data['off_peak']['time_range']],
    'delays': [peak_hours_data['morning']['avg_delay'], peak_hours_data['evening']['avg_delay'], peak_hours_data['off_peak']['avg_delay']],
    'colors': ['#ff6b35', '#f5576c', '#00d084']
}

geographic_chart_data = {
    'labels': [geographic_stats[area]['label'] for area in ['central', 'south', 'north', 'west']],
    'delays': [geographic_stats[area]['avg_delay'] for area in ['central', 'south', 'north', 'west']],
    'colors': [geographic_stats[area]['color'] for area in ['central', 'south', 'north', 'west']]
}

problem_routes_chart_data = {
    'labels': [route['route'] for route in top_problem_routes[:10]],
    'delays': [route['avg_delay'] for route in top_problem_routes[:10]],
    'colors': ['#f5576c' if route['avg_delay'] > 10 else '#ff6b35' for route in top_problem_routes[:10]]
}

peak_morning = df_analyzed.filter((pl.col('hour') >= 7) & (pl.col('hour') < 10))
peak_evening = df_analyzed.filter((pl.col('hour') >= 17) & (pl.col('hour') < 20))
peak_off = df_analyzed.filter(~((pl.col('hour') >= 7) & (pl.col('hour') < 10)) & ~((pl.col('hour') >= 17) & (pl.col('hour') < 20)))

peak_hours_chart_data = {'labels': ['Morning (7-10)', 'Evening (17-20)', 'Off Peak'], 'delays': [round(peak_morning['delay_minutes'].mean(), 1), round(peak_evening['delay_minutes'].mean(), 1), round(peak_off['delay_minutes'].mean(), 1)], 'colors': ['#ff6b35', '#f5576c', '#00d084']}
geographic_chart_data = {'labels': [geographic_stats[area]['label'] for area in ['central', 'south', 'north', 'west']], 'delays': [geographic_stats[area]['avg_delay'] for area in ['central', 'south', 'north', 'west']], 'colors': [geographic_stats[area]['color'] for area in ['central', 'south', 'north', 'west']]}
problem_routes_chart_data = {'labels': [route['route'] for route in top_problem_routes[:10]], 'delays': [route['avg_delay'] for route in top_problem_routes[:10]], 'colors': ['#f5576c' if route['avg_delay'] > 10 else '#ff6b35' for route in top_problem_routes[:10]]}


In [ ]:
# recommendations
best_times = sorted(hourly_data, key=lambda x: x['avg_delay'])[:5]
worst_times = sorted(hourly_data, key=lambda x: x['avg_delay'], reverse=True)[:5]

recommendations = {
    'best_times': [{'hour': f"{data['hour']:02d}:00", 'delay': data['avg_delay'], 'reason': 'Lowest delays'} for data in best_times],
    'worst_times': [{'hour': f"{data['hour']:02d}:00", 'delay': data['avg_delay'], 'reason': 'Highest delays'} for data in worst_times],
    'avoid_routes': top_problem_routes[:3],
    'best_routes': sorted([{'route': k, 'avg_delay': v['directions'][list(v['directions'].keys())[0]]['delay']} for k, v in route_data.items()], key=lambda x: x['avg_delay'])[:5]
}

In [ ]:
# Animation Data
hourly_route_df = df_analyzed.group_by(['route_short_name', 'direction_id', 'hour']).agg(pl.col('delay_minutes').mean().alias('avg_delay')).sort(['route_short_name', 'hour']).to_pandas()
route_hourly_stories = {}
for _, row in hourly_route_df.iterrows():
    key = f"{row['route_short_name']}_{row['direction_id']}"
    if key not in route_hourly_stories: route_hourly_stories[key] = {}
    route_hourly_stories[key][int(row['hour'])] = round(row['avg_delay'], 1)

In [ ]:
import json
import polars as pl

print("Data ready. Generating HTML")

html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Rush Hour Reality - Dublin Bus Analytics</title>
    <script src='https://api.mapbox.com/mapbox-gl-js/v3.0.0/mapbox-gl.js'></script>
    <link href='https://api.mapbox.com/mapbox-gl-js/v3.0.0/mapbox-gl.css' rel='stylesheet' />
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/@turf/turf@6/turf.min.js"></script>

    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        :root { --primary: #00d4ff; --secondary: #ff006e; --accent: #ffbe0b; --dark: #0a0e27; --darker: #050810; --text-light: #ffffff; --text-gray: #b0b0b0; --on-time: #00d084; --minor-delay: #ffb700; --moderate-delay: #ff6b35; --major-delay: #f5576c; }
        html { scroll-behavior: smooth; }
        body { font-family: 'Segoe UI', 'Helvetica Neue', Arial, sans-serif; background: var(--dark); color: var(--text-light); overflow-x: hidden; line-height: 1.6; }
        ::-webkit-scrollbar { width: 8px; } ::-webkit-scrollbar-track { background: var(--dark); } ::-webkit-scrollbar-thumb { background: var(--primary); border-radius: 4px; }

        .hero { position: relative; width: 100%; height: 100vh; background: linear-gradient(135deg, var(--darker) 0%, #1a1f3a 100%); display: flex; align-items: center; justify-content: center; overflow: hidden; z-index: 1; }
        .hero::before { content: ''; position: absolute; top: -50%; right: -50%; width: 200%; height: 200%; background: radial-gradient(circle, rgba(0, 212, 255, 0.15) 0%, transparent 70%); animation: bgFloat 20s ease-in-out infinite; }
        @keyframes bgFloat { 0%, 100% { transform: translate(0, 0); } 50% { transform: translate(50px, 50px); } }
        .hero-content { position: relative; z-index: 2; text-align: center; animation: fadeInUp 1s ease-out; }
        @keyframes fadeInUp { from { opacity: 0; transform: translateY(40px); } to { opacity: 1; transform: translateY(0); } }
        .hero-icon { font-size: 6rem; margin-bottom: 24px; animation: float 3s ease-in-out infinite; }
        @keyframes float { 0%, 100% { transform: translateY(0px); } 50% { transform: translateY(-30px); } }
        .hero h1 { font-size: 4.5rem; font-weight: 900; margin-bottom: 16px; letter-spacing: -2px; background: linear-gradient(135deg, var(--primary), var(--secondary)); -webkit-background-clip: text; -webkit-text-fill-color: transparent; }
        .hero p { font-size: 1.3rem; color: var(--text-gray); margin-bottom: 48px; font-weight: 300; letter-spacing: 2px; }
        .cta-buttons { display: flex; gap: 20px; justify-content: center; flex-wrap: wrap; animation: fadeIn 1.5s ease-out; }
        .btn { padding: 16px 48px; border: none; border-radius: 50px; font-size: 1rem; font-weight: 700; cursor: pointer; transition: all 0.3s ease; text-transform: uppercase; letter-spacing: 1.5px; position: relative; overflow: hidden; }
        .btn-primary { background: linear-gradient(135deg, var(--primary), #00a8cc); color: var(--dark); box-shadow: 0 8px 30px rgba(0, 212, 255, 0.3); }
        .btn-primary:hover { transform: translateY(-3px); box-shadow: 0 12px 40px rgba(0, 212, 255, 0.5); }
        .btn-secondary { background: transparent; border: 2px solid var(--primary); color: var(--primary); }
        .btn-secondary:hover { background: var(--primary); color: var(--dark); transform: translateY(-3px); }
        .scroll-indicator { position: absolute; bottom: 40px; left: 50%; transform: translateX(-50%); animation: bounce 2s infinite; }
        @keyframes bounce { 0%, 100% { transform: translateX(-50%) translateY(0); } 50% { transform: translateX(-50%) translateY(-20px); } }
        .scroll-indicator svg { width: 30px; height: 30px; stroke: var(--primary); fill: none; stroke-width: 2; }

        .about { min-height: auto; background: linear-gradient(135deg, var(--dark) 0%, #1a1f3a 100%); padding: 100px 40px; display: flex; align-items: center; justify-content: center; position: relative; z-index: 3; }
        .about-content { max-width: 1000px; animation: slideInLeft 0.8s ease-out; }
        @keyframes slideInLeft { from { opacity: 0; transform: translateX(-60px); } to { opacity: 1; transform: translateX(0); } }
        .section-title { font-size: 3.5rem; font-weight: 900; margin-bottom: 32px; background: linear-gradient(135deg, var(--primary), var(--secondary)); -webkit-background-clip: text; -webkit-text-fill-color: transparent; }
        .about-text { font-size: 1.2rem; color: var(--text-gray); line-height: 1.8; margin-bottom: 32px; }
        .about-stats { display: grid; grid-template-columns: repeat(3, 1fr); gap: 40px; margin-top: 60px; }
        .stat-box { text-align: center; padding: 30px; background: rgba(0, 212, 255, 0.1); border-radius: 16px; border: 1px solid rgba(0, 212, 255, 0.2); transition: all 0.3s ease; }
        .stat-box:hover { background: rgba(0, 212, 255, 0.2); transform: translateY(-10px); }
        .stat-number { font-size: 3rem; font-weight: 900; color: var(--primary); margin-bottom: 12px; }
        .stat-label { font-size: 0.95rem; color: var(--text-gray); text-transform: uppercase; letter-spacing: 1px; }

        .insights-section { background: linear-gradient(135deg, #1a1f3a 0%, var(--dark) 100%); padding: 100px 40px; position: relative; z-index: 3; }
        .insights-title { font-size: 2.5rem; font-weight: 900; margin-bottom: 60px; text-align: center; background: linear-gradient(135deg, var(--primary), var(--secondary)); -webkit-background-clip: text; -webkit-text-fill-color: transparent; }
        .insights-grid { max-width: 1200px; margin: 0 auto; display: grid; grid-template-columns: repeat(auto-fit, minmax(280px, 1fr)); gap: 40px; }
        .insight-card { background: linear-gradient(135deg, rgba(0, 212, 255, 0.1), rgba(255, 0, 110, 0.05)); border: 2px solid rgba(0, 212, 255, 0.2); border-radius: 16px; padding: 40px; cursor: pointer; transition: all 0.3s ease; position: relative; overflow: hidden; }
        .insight-card:hover { transform: translateY(-15px); border-color: var(--primary); box-shadow: 0 20px 50px rgba(0, 212, 255, 0.2); }
        .insight-icon { font-size: 3rem; margin-bottom: 24px; background: linear-gradient(135deg, var(--primary), var(--secondary)); -webkit-background-clip: text; -webkit-text-fill-color: transparent; }
        .insight-title { font-size: 1.4rem; font-weight: 700; margin-bottom: 16px; color: var(--text-light); }
        .insight-desc { font-size: 0.95rem; color: var(--text-gray); line-height: 1.7; margin-bottom: 20px; }
        .insight-value { font-size: 2rem; font-weight: 900; color: var(--primary); margin-bottom: 8px; }
        .insight-subtext { font-size: 0.8rem; color: rgba(0, 212, 255, 0.7); text-transform: uppercase; letter-spacing: 1px; }

        .dashboard-app { width: 100%; height: 100vh; position: fixed; top: 0; left: 0; background: var(--darker); z-index: 100; display: none; flex-direction: column; }
        .dashboard-app.active { display: flex; }
        #map { width: 100%; height: 100%; flex: 1; }
        .mapboxgl-ctrl-logo, .mapboxgl-ctrl-attrib { display: none !important; }
        .app-header { position: fixed; top: 0; left: 0; right: 0; height: 70px; background: rgba(10, 14, 39, 0.95); backdrop-filter: blur(20px); border-bottom: 1px solid rgba(0, 212, 255, 0.1); display: none; align-items: center; justify-content: space-between; padding: 0 32px; z-index: 1008; }
        .dashboard-app .app-header { display: flex; }
        .app-logo { font-size: 1.5rem; font-weight: 900; background: linear-gradient(135deg, var(--primary), var(--secondary)); -webkit-background-clip: text; -webkit-text-fill-color: transparent; }
        .app-controls button { background: none; border: none; color: var(--primary); font-size: 1.2rem; cursor: pointer; transition: all 0.3s ease; }

        .app-sidebar { position: fixed; left: -420px; top: 70px; width: 400px; height: calc(100vh - 70px); background: linear-gradient(135deg, rgba(10, 14, 39, 0.98), rgba(26, 31, 58, 0.98)); backdrop-filter: blur(20px); border-right: 1px solid rgba(0, 212, 255, 0.2); z-index: 1007; padding: 32px 24px; overflow-y: auto; transition: left 0.4s cubic-bezier(0.34, 1.56, 0.64, 1); }
        .app-sidebar.active { left: 0; }
        .sidebar-header { display: flex; justify-content: space-between; align-items: center; margin-bottom: 32px; border-bottom: 1px solid rgba(0, 212, 255, 0.1); padding-bottom: 16px; }
        .sidebar-title { font-size: 1.5rem; font-weight: 800; color: var(--text-light); }
        .sidebar-close { background: none; border: none; font-size: 1.8rem; cursor: pointer; color: var(--primary); }
        .sidebar-section { margin-bottom: 32px; }
        .section-label { font-size: 0.75rem; font-weight: 900; text-transform: uppercase; color: var(--primary); margin-bottom: 16px; letter-spacing: 2px; }
        .area-grid { display: grid; grid-template-columns: 1fr; gap: 12px; }
        .area-btn, .bus-btn, .stop-btn, .direction-btn { background: linear-gradient(135deg, rgba(0, 212, 255, 0.1), rgba(0, 168, 204, 0.1)); border: 2px solid rgba(0, 212, 255, 0.3); color: var(--primary); padding: 14px 20px; border-radius: 12px; font-weight: 700; cursor: pointer; transition: all 0.3s ease; position: relative; overflow: hidden; margin-bottom: 8px; width: 100%; display: block; text-align: left; }
        .area-btn:hover, .bus-btn:hover, .stop-btn:hover { background: rgba(0, 212, 255, 0.2); border-color: var(--primary); transform: translateY(-2px); }

        .search-section { margin-bottom: 24px; max-height: 0; opacity: 0; overflow: hidden; transition: max-height 0.4s ease, opacity 0.4s ease; }
        .search-section.active { max-height: 1000px; opacity: 1; }
        .search-input { width: 100%; background: linear-gradient(135deg, rgba(0, 212, 255, 0.1), rgba(0, 168, 204, 0.1)); border: 2px solid rgba(0, 212, 255, 0.3); color: var(--primary); padding: 12px 16px; border-radius: 10px; font-weight: 700; margin-bottom: 16px; }

        .bus-grid, .stop-grid { display: grid; gap: 10px; max-height: 0; opacity: 0; overflow: hidden; transition: max-height 0.3s ease, opacity 0.3s ease; }
        .bus-grid { grid-template-columns: repeat(2, 1fr); }
        .stop-grid { grid-template-columns: 1fr; }
        .bus-grid.active, .stop-grid.active { max-height: 300px; opacity: 1; }

        .direction-view { display: none; }
        .direction-view.active { display: block; }
        .back-btn { background: linear-gradient(135deg, var(--primary), #00a8cc); border: none; color: var(--dark); padding: 12px 20px; border-radius: 10px; font-weight: 700; cursor: pointer; width: 100%; margin-bottom: 24px; }

        .route-card { position: fixed; bottom: 40px; left: 40px; background: linear-gradient(135deg, rgba(10, 14, 39, 0.95), rgba(26, 31, 58, 0.95)); backdrop-filter: blur(20px); border: 2px solid var(--primary); padding: 32px; border-radius: 16px; z-index: 1006; display: none; min-width: 350px; }
        .route-card.active { display: block; animation: slideUp 0.4s; }
        @keyframes slideUp { from { opacity: 0; transform: translateY(30px); } to { opacity: 1; transform: translateY(0); } }
        .route-badge { display: inline-block; background: linear-gradient(135deg, var(--primary), #00a8cc); color: var(--dark); padding: 8px 20px; border-radius: 25px; font-size: 0.8rem; font-weight: 900; margin-bottom: 16px; text-transform: uppercase; }
        .route-name { font-size: 2.5rem; font-weight: 900; color: var(--text-light); margin-bottom: 12px; background: linear-gradient(135deg, var(--primary), var(--secondary)); -webkit-background-clip: text; -webkit-text-fill-color: transparent; }
        .route-delay { font-size: 1.1rem; color: var(--text-gray); margin-bottom: 24px; }
        .route-stats { display: grid; grid-template-columns: repeat(2, 1fr); gap: 16px; border-top: 1px solid rgba(0, 212, 255, 0.2); padding-top: 24px; }
        .stat-value { font-size: 1.8rem; font-weight: 900; background: linear-gradient(135deg, var(--primary), var(--secondary)); -webkit-background-clip: text; -webkit-text-fill-color: transparent; }
        .stat-label { font-size: 0.75rem; color: var(--text-gray); text-transform: uppercase; margin-top: 8px; }

        .delay-legend { position: fixed; bottom: 100px; right: 20px; background: rgba(10, 14, 39, 0.95); border: 2px solid var(--primary); border-radius: 12px; padding: 16px; z-index: 1005; min-width: 220px; backdrop-filter: blur(20px); display: none; }
        .delay-legend.active { display: block; }
        .legend-title { font-weight: 900; color: var(--primary); margin-bottom: 12px; font-size: 0.85rem; text-transform: uppercase; }
        .legend-item { display: flex; align-items: center; gap: 8px; margin-bottom: 8px; }
        .legend-dot { width: 16px; height: 16px; border-radius: 50%; border: 2px solid #fff; }

        .modal-fullscreen { position: fixed; inset: 0; background: rgba(0,0,0,0.9); z-index: 2000; display: none; align-items: center; justify-content: center; }
        .modal-fullscreen.active { display: flex; }
        .modal-fullscreen-content { background: linear-gradient(135deg, rgba(10, 14, 39, 0.99), rgba(26, 31, 58, 0.99)); border: 3px solid var(--primary); border-radius: 24px; padding: 40px; width: 100%; max-width: 1000px; height: 85vh; overflow-y: auto; position: relative; }
        .modal-fullscreen-close { position: absolute; top: 30px; right: 30px; background: var(--primary); border: none; width: 40px; height: 40px; border-radius: 50%; font-weight: 900; cursor: pointer; }
        .chart-container-large { position: relative; height: 400px; margin-bottom: 30px; background: rgba(0, 212, 255, 0.05); padding: 20px; border-radius: 16px; border: 1px solid rgba(0, 212, 255, 0.1); }
        .recom-section { padding: 30px; background: rgba(0, 212, 255, 0.05); border: 1px solid rgba(0, 212, 255, 0.1); border-radius: 12px; margin: 30px 0; }
        .recom-title { color: var(--primary); font-weight: 900; margin-bottom: 20px; font-size: 1.1rem; text-transform: uppercase; }
        .rec-grid { display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin: 20px 0; }
        .rec-box { background: rgba(255, 0, 110, 0.1); border: 1px solid rgba(255, 0, 110, 0.3); border-radius: 10px; padding: 16px; }
        .rec-box.best { background: rgba(0, 208, 132, 0.1); border-color: rgba(0, 208, 132, 0.3); }
        .rec-hour { color: var(--primary); font-weight: 900; font-size: 1.1rem; }
        .rec-delay { color: var(--text-gray); font-size: 0.9rem; margin-top: 5px; }

        .toggle-sidebar { position: fixed; top: 90px; left: 24px; background: linear-gradient(135deg, var(--primary), #00a8cc); border: none; color: var(--dark); padding: 14px 24px; border-radius: 50px; font-size: 1.1rem; cursor: pointer; z-index: 1005; display: none; font-weight: 900; text-transform: uppercase; }
        .dashboard-app .toggle-sidebar { display: block; }

        .modal-backdrop { position: fixed; top: 0; left: 0; right: 0; bottom: 0; background: rgba(0,0,0,0.7); z-index: 1010; display: none; }
        .modal-backdrop.active { display: block; }
        .stop-modal { position: fixed; top: 50%; left: 50%; transform: translate(-50%, -50%); background: linear-gradient(135deg, rgba(10, 14, 39, 0.98), rgba(26, 31, 58, 0.98)); border: 3px solid var(--primary); border-radius: 24px; padding: 32px; color: var(--text-light); z-index: 1011; display: none; width: 90%; max-width: 500px; max-height: 80vh; overflow-y: auto; }
        .stop-modal.active { display: block; }
        .stop-modal .modal-header { display: flex; justify-content: space-between; align-items: center; margin-bottom: 24px; border-bottom: 2px solid rgba(0, 212, 255, 0.2); padding-bottom: 16px; }
        .stop-modal .modal-title { font-size: 1.4rem; font-weight: 900; }
        .stop-modal .modal-close { background: none; border: none; font-size: 1.5rem; color: var(--primary); cursor: pointer; }
        .modal-buses { display: grid; gap: 12px; }
        .modal-bus-btn { background: linear-gradient(135deg, rgba(0, 212, 255, 0.1), rgba(0, 168, 204, 0.1)); border: 2px solid rgba(0, 212, 255, 0.3); color: var(--primary); padding: 14px 16px; border-radius: 12px; font-weight: 700; cursor: pointer; text-align: left; display: flex; justify-content: space-between; align-items: center; }
        .modal-bus-btn:hover { background: rgba(0, 212, 255, 0.2); border-color: var(--primary); transform: translateX(8px); }
        .modal-bus-name { font-size: 1rem; font-weight: 900; }
        .modal-bus-delay { font-size: 0.8rem; background: rgba(255, 190, 11, 0.2); padding: 4px 10px; border-radius: 6px; }

        .stop-tooltip { position: absolute; background: rgba(10, 14, 39, 0.95); border: 2px solid var(--primary); padding: 16px; border-radius: 12px; z-index: 2000; pointer-events: none; display: none; backdrop-filter: blur(10px); min-width: 200px; box-shadow: 0 10px 40px rgba(0, 0, 0, 0.5); }
        .stop-tooltip.active { display: block; }
        .tooltip-title { color: var(--primary); font-weight: 900; margin-bottom: 5px; font-size: 1rem; }
        .tooltip-info { color: var(--text-light); font-size: 0.9rem; margin-bottom: 3px; }

        .analytics-hud { position: fixed; top: 90px; right: 20px; width: 340px; background: rgba(10, 14, 39, 0.95); border: 1px solid var(--primary); border-left: 4px solid var(--secondary); border-radius: 12px; padding: 20px; z-index: 1006; display: none; backdrop-filter: blur(15px); box-shadow: 0 10px 40px rgba(0,0,0,0.6); animation: slideInRight 0.5s; }
        .analytics-hud.active { display: block; }
        .time-controls { position: fixed; bottom: 40px; left: 50%; transform: translateX(-50%); width: 600px; background: rgba(10, 14, 39, 0.95); border: 1px solid var(--primary); border-radius: 50px; padding: 15px 30px; display: none; align-items: center; gap: 20px; z-index: 1006; backdrop-filter: blur(10px); }
        .time-controls.active { display: flex; animation: slideUp 0.5s; }
        .play-btn { width: 45px; height: 45px; border-radius: 50%; background: var(--primary); border: none; cursor: pointer; font-size: 1.2rem; display: flex; align-items: center; justify-content: center; }
        input[type=range] { flex: 1; accent-color: var(--primary); cursor: pointer; }
        .clock { font-family: monospace; font-size: 1.5rem; color: var(--primary); font-weight: 900; min-width: 90px; text-align: center; }
        @keyframes slideInRight { from { transform: translateX(50px); opacity: 0; } to { transform: translateX(0); opacity: 1; } }
    </style>
</head>
<body>
    <div id="website-sections">
        <section class="hero" id="hero">
            <div class="hero-content">
                <div class="hero-icon">🚌</div>
                <h1>RUSH HOUR REALITY</h1>
                <p>Dublin Bus Delays Intelligence Platform</p>
                <div class="cta-buttons">
                    <button class="btn btn-primary" onclick="launchDashboard()">Explore Dashboard</button>
                    <button class="btn btn-secondary" onclick="scrollToSection('insights')">Learn More</button>
                </div>
            </div>
            <div class="scroll-indicator"><svg viewBox="0 0 24 24"><polyline points="18 15 12 21 6 15"></polyline></svg></div>
        </section>

        <section class="about" id="about">
            <div class="about-content">
                <h2 class="section-title">What is Rush Hour Reality?</h2>
                <p class="about-text">Rush Hour Reality is an advanced data analytics platform designed to visualize and predict Dublin bus delays in real-time.</p>
                <div class="about-stats">
                    <div class="stat-box"><div class="stat-number">""" + str(overall_stats['total_routes']) + """</div><div class="stat-label">Bus Routes</div></div>
                    <div class="stat-box"><div class="stat-number">""" + str(overall_stats['total_stops']) + """</div><div class="stat-label">Bus Stops</div></div>
                    <div class="stat-box"><div class="stat-number">""" + str(overall_stats['overall_avg_delay']) + """m</div><div class="stat-label">Avg Delay</div></div>
                </div>
            </div>
        </section>

        <section class="insights-section" id="insights">
            <h2 class="insights-title">📊 Dublin's Transport Challenges</h2>
            <div class="insights-grid">
                <div class="insight-card" onclick="openFullscreenModal('peakHoursModal')">
                    <div class="insight-icon">⏰</div>
                    <h3 class="insight-title">Peak Hour Delays</h3>
                    <div class="insight-value">""" + str(peak_hours_data['morning']['avg_delay']) + """m</div>
                    <div class="insight-subtext">Morning Peak Average</div>
                </div>
                <div class="insight-card" onclick="openFullscreenModal('problemRoutesModal')">
                    <div class="insight-icon">🚍</div>
                    <h3 class="insight-title">Problem Routes</h3>
                    <div class="insight-value">""" + str(top_problem_routes[0]['route']) + """</div>
                    <div class="insight-subtext">Route with highest delays</div>
                </div>
                <div class="insight-card" onclick="openFullscreenModal('geographicModal')">
                    <div class="insight-icon">🗺️</div>
                    <h3 class="insight-title">Geographic Hotspots</h3>
                    <div class="insight-value">""" + str(geographic_stats['central']['avg_delay']) + """m</div>
                    <div class="insight-subtext">Central Dublin Average</div>
                </div>
                <div class="insight-card" onclick="openFullscreenModal('dailyPatternModal')">
                    <div class="insight-icon">📈</div>
                    <h3 class="insight-title">Daily Pattern</h3>
                    <div class="insight-value">""" + str(max(hourly_data, key=lambda x: x['avg_delay'])['avg_delay']) + """m</div>
                    <div class="insight-subtext">Peak delay hour</div>
                </div>
                <div class="insight-card" onclick="openFullscreenModal('recomModal')">
                    <div class="insight-icon">⭐</div>
                    <h3 class="insight-title">Smart Recommendations</h3>
                    <div class="insight-value">Live</div>
                    <div class="insight-subtext">Recommendations</div>
                </div>
            </div>
        </section>
    </div>

    <div class="modal-fullscreen" id="peakHoursModal"><div class="modal-fullscreen-content"><button class="modal-fullscreen-close" onclick="closeFullscreenModal('peakHoursModal')">✕</button><h2 class="modal-fullscreen-title">⏰ Peak Hour Delays</h2><div class="chart-container-large"><canvas id="peakHoursChart"></canvas></div></div></div>
    <div class="modal-fullscreen" id="problemRoutesModal"><div class="modal-fullscreen-content"><button class="modal-fullscreen-close" onclick="closeFullscreenModal('problemRoutesModal')">✕</button><h2 class="modal-fullscreen-title">🚍 Problem Routes</h2><div class="chart-container-large"><canvas id="problemRoutesChart"></canvas></div></div></div>
    <div class="modal-fullscreen" id="geographicModal"><div class="modal-fullscreen-content"><button class="modal-fullscreen-close" onclick="closeFullscreenModal('geographicModal')">✕</button><h2 class="modal-fullscreen-title">🗺️ Geographic Hotspots</h2><div class="chart-container-large"><canvas id="geographicChart"></canvas></div></div></div>
    <div class="modal-fullscreen" id="dailyPatternModal"><div class="modal-fullscreen-content"><button class="modal-fullscreen-close" onclick="closeFullscreenModal('dailyPatternModal')">✕</button><h2 class="modal-fullscreen-title">📈 Daily Pattern</h2><div class="chart-container-large"><canvas id="dailyPatternChart"></canvas></div></div></div>

    <div class="modal-fullscreen" id="recomModal">
        <div class="modal-fullscreen-content">
            <button class="modal-fullscreen-close" onclick="closeFullscreenModal('recomModal')">✕</button>
            <h2 class="modal-fullscreen-title">⭐ Smart Recommendations</h2>
            <div class="recom-section"><div class="recom-title">🟢 Best Times to Travel</div><div class="rec-grid">""" + ''.join([f'<div class="rec-box best"><div class="rec-hour">{t["hour"]}</div><div class="rec-delay">Avg: {t["delay"]}m</div></div>' for t in recommendations['best_times']]) + """</div></div>
            <div class="recom-section"><div class="recom-title">🔴 Worst Times (Avoid)</div><div class="rec-grid">""" + ''.join([f'<div class="rec-box"><div class="rec-hour">{t["hour"]}</div><div class="rec-delay">Avg: {t["delay"]}m</div></div>' for t in recommendations['worst_times']]) + """</div></div>
            <div class="recom-section"><div class="recom-title">🚫 Routes to Avoid</div><div class="rec-grid">""" + ''.join([f'<div class="rec-box"><div class="rec-hour">Route {r["route"]}</div><div class="rec-delay">{r["avg_delay"]}m avg</div></div>' for r in recommendations['avoid_routes']]) + """</div></div>
            <div class="recom-section"><div class="recom-title">✅ Best Routes</div><div class="rec-grid">""" + ''.join([f'<div class="rec-box best"><div class="rec-hour">Route {r["route"]}</div><div class="rec-delay">{r["avg_delay"]}m avg</div></div>' for r in recommendations['best_routes']]) + """</div></div>
        </div>
    </div>

    <div class="dashboard-app" id="dashboard-app">
        <div class="app-header">
            <div class="app-logo">🚌 Rush Hour Reality</div>
            <div class="app-controls"><button onclick="exitDashboard()">✕ Exit</button></div>
        </div>

        <div id="map"></div>
        <div id="stopTooltip" class="stop-tooltip">
            <div id="tooltipTitle" class="tooltip-title"></div>
            <div id="tooltipStop" class="tooltip-info"></div>
            <div id="tooltipDelay" class="tooltip-info"></div>
        </div>

        <div class="delay-legend active" id="delayLegend">
            <div class="legend-title">Delay Severity</div>
            <div class="legend-item"><div class="legend-dot" style="background: var(--on-time);"></div> On Time</div>
            <div class="legend-item"><div class="legend-dot" style="background: var(--minor-delay);"></div> Minor</div>
            <div class="legend-item"><div class="legend-dot" style="background: var(--moderate-delay);"></div> Moderate</div>
            <div class="legend-item"><div class="legend-dot" style="background: var(--major-delay);"></div> Major</div>
        </div>

        <div class="analytics-hud" id="analyticsHud">
            <h3 class="hud-title">LIVE METRICS</h3>
            <div style="display:flex; justify-content:space-between; margin-bottom:10px;">
                <div id="hudStatus" style="font-weight:bold; font-size:1.2rem; color:var(--on-time);">ON TIME</div>
                <div id="hudSpeed" style="color:#ccc; font-size:1.2rem;">30 km/h</div>
            </div>
            <div style="font-size:0.9rem; color:#ccc; line-height:1.4;" id="hudDesc">Select a route.</div>
            <div style="margin-top:15px; border-top:1px solid rgba(255,255,255,0.1); padding-top:10px;">
                <div class="hud-title">24H TREND</div>
                <div style="height:80px; width:100%;"><canvas id="sparklineChart"></canvas></div>
            </div>
        </div>
        <div class="time-controls" id="timeControls">
            <button class="play-btn" id="playBtn" onclick="togglePlay()"><i class="fas fa-play"></i></button>
            <input type="range" id="timeSlider" min="300" max="1439" value="420" step="15" oninput="manualTime(this.value)">
            <div class="clock" id="clock">07:00</div>
        </div>

        <button class="toggle-sidebar" id="toggleBtn" onclick="toggleSidebar()">☰ Routes</button>

        <div class="app-sidebar" id="sidebar">
            <div class="sidebar-header"><h3 class="sidebar-title">🗺️ Routes</h3><button class="sidebar-close" onclick="toggleSidebar()">×</button></div>
            <div id="areaView">
                <div class="sidebar-section">
                    <div class="section-label">Select Area</div>
                    <div class="area-grid">
                        <button class="area-btn" onclick="selectArea('north', this)">North Dublin</button>
                        <button class="area-btn" onclick="selectArea('central', this)">Central Dublin</button>
                        <button class="area-btn" onclick="selectArea('south', this)">South Dublin</button>
                        <button class="area-btn" onclick="selectArea('west', this)">West Dublin</button>
                    </div>
                </div>

                <div class="sidebar-section search-section" id="busSearchSection">
                    <div class="section-label">Search Buses</div>
                    <input type="text" class="search-input" id="busSearchInput" placeholder="Type E1, E, 4..." oninput="filterBuses()">
                    <div class="bus-grid" id="busGrid"></div>
                </div>

                <div class="sidebar-section search-section" id="stopSearchSection">
                    <div class="section-label">Search Bus Stops</div>
                    <input type="text" class="search-input" id="stopSearchInput" placeholder="Type D, O, Street..." oninput="filterStops()">
                    <div class="stop-grid" id="stopGrid"></div>
                </div>
            </div>

            <div id="directionView" class="direction-view">
                <button class="back-btn" onclick="goBack()">← Back to Buses</button>
                <div class="sidebar-section"><div class="section-label" id="directionBusLabel"></div><div class="direction-grid" id="directionGrid"></div></div>
            </div>
        </div>

        <div class="route-card" id="routeCard">
            <div class="route-badge" id="routeBadge"></div>
            <h3 class="route-name" id="routeName"></h3>
            <p class="route-delay">Average Delay: <span class="route-delay-value" id="routeDelay"></span></p>
            <div class="route-stats">
                <div class="stat"><div class="stat-value" id="statStops">0</div><div class="stat-label">Stops</div></div>
                <div class="stat"><div class="stat-value" id="statTendency">-</div><div class="stat-label">Status</div></div>
            </div>
        </div>

        <div class="modal-backdrop" id="modalBackdrop" onclick="closeStopModal()"></div>
        <div class="stop-modal" id="stopModal">
            <div class="modal-header"><div class="modal-title" id="modalStopName">Stop Name</div><button class="modal-close" onclick="closeStopModal()">✕</button></div>
            <div class="modal-buses" id="modalBusesList"></div>
        </div>
    </div>

    <script>
        mapboxgl.accessToken = 'pk.eyJ1IjoiYXJ1bjRnbXIiLCJhIjoiY21pa3NkZjRoMDBnNjNjc2RwOWhhN3VndSJ9.apzM1q8wl0-QqVl2HrD9ng';
        const routeData = """ + json.dumps(route_data) + """;
        const areaBuses = """ + json.dumps(area_buses) + """;
        const peakHoursChartData = """ + json.dumps(peak_hours_chart_data) + """;
        const geographicChartData = """ + json.dumps(geographic_chart_data) + """;
        const problemRoutesChartData = """ + json.dumps(problem_routes_chart_data) + """;
        const hourlyData = """ + json.dumps(hourly_data) + """;
        const routeStories = """ + json.dumps(route_hourly_stories) + """;

        let map, currentRouteLayers = [], selectedBus = null, dashboardActive = false, currentBusRoute = null, currentArea = null;
        let allBuses = [], allStops = [], charts = {};
        let isPlaying=false, animationId, timeInterval, activeGeojson=null, busMarker=null, progress=0, currentSpeed=0.0005, sparklineChart;

        function openFullscreenModal(modalId) {
            document.getElementById(modalId).classList.add('active');
            if (modalId === 'peakHoursModal' && !charts['peakHours']) initPeakHoursChart();
            else if (modalId === 'problemRoutesModal' && !charts['problemRoutes']) initProblemRoutesChart();
            else if (modalId === 'geographicModal' && !charts['geographic']) initGeographicChart();
            else if (modalId === 'dailyPatternModal' && !charts['dailyPattern']) initDailyPatternChart();
        }
        function closeFullscreenModal(modalId) { document.getElementById(modalId).classList.remove('active'); }
        function scrollToSection(id) { document.getElementById(id).scrollIntoView({ behavior: 'smooth' }); }

        function launchDashboard() {
            document.getElementById('website-sections').style.display = 'none';
            document.getElementById('dashboard-app').classList.add('active');
            dashboardActive = true;
            setTimeout(() => { if (!map) { initMap(); document.getElementById('sidebar').classList.add('active'); } }, 100);
        }

        function exitDashboard() {
            document.getElementById('dashboard-app').classList.remove('active');
            document.getElementById('website-sections').style.display = 'block';
            dashboardActive = false; stopSim(); window.scrollTo({ top: 0, behavior: 'smooth' });
        }

        function initMap() {
            if (map) return;
            map = new mapboxgl.Map({ container: 'map', style: 'mapbox://styles/mapbox/dark-v11', center: [-6.2603, 53.3498], zoom: 12, pitch: 60, bearing: -17.6 });
            map.on('load', () => {
                map.addLayer({ 'id': '3d-buildings', 'source': 'composite', 'source-layer': 'building', 'type': 'fill-extrusion', 'minzoom': 14, 'paint': { 'fill-extrusion-color': '#1a1f3a', 'fill-extrusion-height': ['interpolate', ['linear'], ['zoom'], 15, 0, 15.05, ['get', 'height']], 'fill-extrusion-opacity': 0.8 } });
            });
            initSparkline();
        }

        function selectArea(area, btn) {
            currentArea = area;
            document.querySelectorAll('.area-btn').forEach(b => b.classList.remove('active'));
            btn.classList.add('active');

            // BUSES
            document.getElementById('busSearchInput').value = '';
            allBuses = areaBuses[area] || [];
            document.getElementById('busSearchSection').classList.add('active');
            displayBuses(allBuses);

            // STOPS (RESTORED)
            document.getElementById('stopSearchInput').value = '';
            allStops = [];
            allBuses.forEach(bus => {
                const directions = routeData[bus].directions;
                Object.keys(directions).forEach(dirId => {
                    const dir = directions[dirId];
                    dir.stops.forEach(stop => {
                        if (!allStops.find(s => s.lat === stop.lat && s.lon === stop.lon)) allStops.push(stop);
                    });
                });
            });
            document.getElementById('stopSearchSection').classList.add('active');
            displayStops(allStops);
        }

        function displayBuses(buses) {
            const busGrid = document.getElementById('busGrid'); busGrid.innerHTML = ''; busGrid.classList.add('active');
            buses.forEach(bus => { const btn = document.createElement('button'); btn.className = 'bus-btn'; btn.textContent = bus; btn.onclick = () => selectBus(bus); busGrid.appendChild(btn); });
        }
        function displayStops(stops) {
            const stopGrid = document.getElementById('stopGrid'); stopGrid.innerHTML = ''; stopGrid.classList.add('active');
            stops.slice(0, 100).forEach(stop => { // Limit initial render for perf
                const btn = document.createElement('button'); btn.className = 'stop-btn'; btn.textContent = stop.name; btn.onclick = () => showStopsAtStop(stop); stopGrid.appendChild(btn);
            });
        }

        function filterBuses() {
            const searchTerm = document.getElementById('busSearchInput').value.toUpperCase();
            const filtered = searchTerm === '' ? allBuses : allBuses.filter(bus => bus.toUpperCase().includes(searchTerm)); displayBuses(filtered);
        }
        function filterStops() {
            const searchTerm = document.getElementById('stopSearchInput').value.toUpperCase();
            const filtered = searchTerm === '' ? allStops : allStops.filter(stop => stop.name.toUpperCase().includes(searchTerm)); displayStops(filtered);
        }

        function selectBus(bus) { selectedBus = bus; showDirectionSelection(bus, routeData[bus].directions); }
        function showDirectionSelection(bus, directions) {
            document.getElementById('areaView').style.display = 'none'; document.getElementById('directionView').classList.add('active'); document.getElementById('directionBusLabel').textContent = `Route ${bus}`;
            const directionGrid = document.getElementById('directionGrid'); directionGrid.innerHTML = '';
            Object.keys(directions).forEach(dirId => { const btn = document.createElement('button'); btn.className = 'direction-btn'; btn.textContent = directions[dirId].name; btn.onclick = () => showRoute(bus, dirId); directionGrid.appendChild(btn); });
        }
        function goBack() { document.getElementById('areaView').style.display = 'block'; document.getElementById('directionView').classList.remove('active'); }
        function toggleSidebar() { document.getElementById('sidebar').classList.toggle('active'); }

        function initPeakHoursChart() { charts['peakHours'] = new Chart(document.getElementById('peakHoursChart').getContext('2d'), { type: 'bar', data: { labels: peakHoursChartData.labels, datasets: [{ label: 'Avg Delay', data: peakHoursChartData.delays, backgroundColor: peakHoursChartData.colors }] }, options: { indexAxis: 'y', responsive: true, maintainAspectRatio: false } }); }
        function initProblemRoutesChart() { charts['problemRoutes'] = new Chart(document.getElementById('problemRoutesChart').getContext('2d'), { type: 'bar', data: { labels: problemRoutesChartData.labels, datasets: [{ label: 'Avg Delay', data: problemRoutesChartData.delays, backgroundColor: problemRoutesChartData.colors }] }, options: { indexAxis: 'y', responsive: true, maintainAspectRatio: false } }); }
        function initGeographicChart() { charts['geographic'] = new Chart(document.getElementById('geographicChart').getContext('2d'), { type: 'doughnut', data: { labels: geographicChartData.labels, datasets: [{ data: geographicChartData.delays, backgroundColor: geographicChartData.colors }] }, options: { responsive: true, maintainAspectRatio: false } }); }

        function initDailyPatternChart() {
            const ctx = document.getElementById('dailyPatternChart').getContext('2d');
            const hours = hourlyData.map(d => d.hour + ':00');
            const delays = hourlyData.map(d => d.avg_delay);
            charts['dailyPattern'] = new Chart(ctx, {
                type: 'line',
                data: {
                    labels: hours,
                    datasets: [{
                        label: 'Average Delay (minutes)',
                        data: delays,
                        borderColor: '#00d4ff',
                        backgroundColor: 'rgba(0, 212, 255, 0.1)',
                        borderWidth: 3,
                        pointRadius: 5,
                        pointBackgroundColor: '#ff006e',
                        pointBorderColor: '#fff',
                        pointBorderWidth: 2,
                        tension: 0.4,
                        fill: true
                    }]
                },
                options: { responsive: true, maintainAspectRatio: false, plugins: { legend: { labels: { color: '#b0b0b0' } } }, scales: { y: { beginAtZero: true, ticks: { color: '#b0b0b0' }, grid: { color: 'rgba(0, 212, 255, 0.1)' } }, x: { ticks: { color: '#b0b0b0' }, grid: { color: 'rgba(0, 212, 255, 0.1)' } } } }
            });
        }

        function showRoute(bus, dirId) {
            const direction = routeData[bus].directions[dirId]; currentBusRoute = { bus, dirId, direction };
            if(busMarker) busMarker.remove();
            currentRouteLayers.forEach(id => { if (map.getLayer(id)) map.removeLayer(id); if (map.getSource(id)) map.removeSource(id); }); currentRouteLayers = [];

            document.getElementById('areaView').style.display = 'block'; document.getElementById('directionView').classList.remove('active'); document.getElementById('sidebar').classList.remove('active'); document.getElementById('toggleBtn').style.display = 'block';
            document.getElementById('routeBadge').textContent = `Route ${bus}`; document.getElementById('routeName').textContent = bus; document.getElementById('routeDelay').textContent = direction.delay + ' min'; document.getElementById('statStops').textContent = direction.stops.length; document.getElementById('statTendency').textContent = direction.category; document.getElementById('routeCard').classList.add('active');

            document.getElementById('analyticsHud').classList.add('active'); document.getElementById('timeControls').classList.add('active'); updateSparkline(`${bus}_${dirId}`);

            const latlngs = direction.stops.map(s => [s.lon, s.lat]); activeGeojson = turf.lineString(latlngs);
            map.addSource(`route-${bus}`, { type: 'geojson', data: { type: 'Feature', geometry: { type: 'LineString', coordinates: latlngs } } });
            map.addLayer({ id: `route-layer-${bus}`, type: 'line', source: `route-${bus}`, paint: { 'line-color': direction.color, 'line-width': 5, 'line-opacity': 0.9 } });
            currentRouteLayers.push(`route-layer-${bus}`); currentRouteLayers.push(`route-${bus}`);

            direction.stops.forEach((stop, i) => {
                const sId = `stop-${i}`;
                map.addSource(sId, { type: 'geojson', data: { type: 'Feature', geometry: { type: 'Point', coordinates: [stop.lon, stop.lat] } } });
                map.addLayer({ id: sId, type: 'circle', source: sId, paint: { 'circle-radius': 6, 'circle-color': stop.color, 'circle-stroke-width': 2, 'circle-stroke-color': '#fff' } });
                currentRouteLayers.push(sId);

                map.on('mouseenter', sId, (e) => {
                    map.getCanvas().style.cursor = 'pointer';
                    const tooltip = document.getElementById('stopTooltip');
                    document.getElementById('tooltipTitle').textContent = stop.name;
                    document.getElementById('tooltipStop').textContent = `Stop`;
                    document.getElementById('tooltipDelay').innerHTML = `Delay: <strong>${stop.delay}m</strong>`;

                    const rect = document.getElementById('map').getBoundingClientRect();
                    const point = map.project([stop.lon, stop.lat]);
                    tooltip.style.left = (rect.left + point.x + 15) + 'px';
                    tooltip.style.top = (rect.top + point.y - 15) + 'px';
                    tooltip.classList.add('active');
                });
                map.on('mouseleave', sId, () => { map.getCanvas().style.cursor = ''; document.getElementById('stopTooltip').classList.remove('active'); });
                map.on('click', sId, () => { showStopsAtStop(stop); });
            });

            const bounds = new mapboxgl.LngLatBounds(); latlngs.forEach(c => bounds.extend(c)); map.fitBounds(bounds, { padding: 100 });
            const el = document.createElement('div'); el.innerHTML = '🚌'; el.style.fontSize = '24px'; busMarker = new mapboxgl.Marker(el).setLngLat(latlngs[0]).addTo(map);
            manualTime(420); progress = 0; animateBus();
        }

        function showStopsAtStop(stop) {
            let buses = [];
            Object.keys(routeData).forEach(bKey => {
                Object.keys(routeData[bKey].directions).forEach(dKey => {
                    const dir = routeData[bKey].directions[dKey];
                    const found = dir.stops.find(s => s.lat === stop.lat && s.lon === stop.lon);
                    if(found) buses.push({bus: bKey, dir: dKey, delay: found.delay});
                });
            });
            buses = buses.filter((v,i,a)=>a.findIndex(t=>(t.bus===v.bus && t.dir===v.dir))===i).sort((a,b)=>a.bus.localeCompare(b.bus));

            document.getElementById('modalStopName').textContent = stop.name;
            const list = document.getElementById('modalBusesList'); list.innerHTML = '';
            buses.forEach(b => {
                const btn = document.createElement('div'); btn.className = 'modal-bus-btn';
                btn.innerHTML = `<span class="modal-bus-name">${b.bus}</span><span class="modal-bus-delay">${b.delay}m</span>`;
                btn.onclick = () => { closeStopModal(); showRoute(b.bus, b.dir); };
                list.appendChild(btn);
            });
            document.getElementById('modalBackdrop').classList.add('active'); document.getElementById('stopModal').classList.add('active');
        }
        function closeStopModal() { document.getElementById('modalBackdrop').classList.remove('active'); document.getElementById('stopModal').classList.remove('active'); }

        function animateBus() {
            if(animationId) cancelAnimationFrame(animationId);
            function loop() {
                progress += currentSpeed; if(progress > 1) progress = 0;
                const pt = turf.along(activeGeojson, progress * turf.length(activeGeojson));
                busMarker.setLngLat(pt.geometry.coordinates);
                animationId = requestAnimationFrame(loop);
            }
            loop();
        }
        function manualTime(mins) {
            mins = parseInt(mins); document.getElementById('timeSlider').value = mins;
            const h = Math.floor(mins/60); const m = mins%60; document.getElementById('clock').innerText = `${h<10?'0'+h:h}:${m<10?'0'+m:m}`;
            const key = `${currentBusRoute.bus}_${currentBusRoute.dirId}`;
            const delay = routeStories[key] ? (routeStories[key][h] || 0) : 0;
            const hasService = routeStories[key] && routeStories[key][h] !== undefined;
            const statusEl = document.getElementById('hudStatus');

            if(!hasService) {
                currentSpeed = 0; statusEl.innerText = "NO SERVICE"; statusEl.style.color="#666"; busMarker.getElement().style.opacity = 0.2;
            } else {
                busMarker.getElement().style.opacity = 1;
                if(delay <= 2) { currentSpeed = 0.0008; statusEl.innerText = "ON TIME"; statusEl.style.color = "#00d084"; }
                else if(delay <= 8) { currentSpeed = 0.0004; statusEl.innerText = "DELAYED SLIGHTLY"; statusEl.style.color = "#ffb700"; }
                else { currentSpeed = 0.0001; statusEl.innerText = "BADLY DELAYED"; statusEl.style.color = "#f5576c"; }
                document.getElementById('hudSpeed').innerText = Math.round(currentSpeed*40000) + " km/h";
            }
        }
        function togglePlay() {
            const btn = document.getElementById('playBtn'); const slider = document.getElementById('timeSlider');
            if(isPlaying) { isPlaying=false; clearInterval(timeInterval); btn.innerHTML='<i class="fas fa-play"></i>'; }
            else { isPlaying=true; btn.innerHTML='<i class="fas fa-pause"></i>'; timeInterval=setInterval(()=>{ let v=parseInt(slider.value); if(v>=1439)v=300; else v+=15; manualTime(v); }, 1000); }
        }
        function stopSim() { isPlaying=false; clearInterval(timeInterval); if(animationId) cancelAnimationFrame(animationId); }
        function initSparkline() { sparklineChart = new Chart(document.getElementById('sparklineChart').getContext('2d'), { type: 'line', data: { labels: [], datasets: [{ data: [], borderColor: '#00d4ff', borderWidth: 2, pointRadius: 0, fill: true, backgroundColor: 'rgba(0,212,255,0.1)' }] }, options: { responsive: true, maintainAspectRatio: false, plugins: { legend: { display: false } }, scales: { x: { display: false }, y: { display: false } } } }); }
        function updateSparkline(key) { const data = routeStories[key] || {}; const lbls = Object.keys(data).sort((a,b)=>a-b); sparklineChart.data.labels = lbls; sparklineChart.data.datasets[0].data = lbls.map(k=>data[k]); sparklineChart.update(); }
    </script>
</body>
</html>
"""

with open('/content/drive/MyDrive/Rush-Hour-Reality/final_submission.html', 'w') as f:
    f.write(html_content)